In [1]:
import koda
from matplotlib import pyplot as plt
import cv2
import numpy as np
import math
%matplotlib inline

in_path = 'dataset'
out_path = 'out'
plt.figure(figsize=(20,20))

# Read image
filename = "IMG_%d.jpg" % 5
img = cv2.imread("%s/%s" % (in_path, filename), cv2.IMREAD_COLOR)
#plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#plt.show()

# Load image into koda engine
doc, imgs = koda.load(img)

# Show koda pipeline
#fig, axes = plt.subplots(1, 4, figsize=(20, 20))
#axes[0].imshow(cv2.cvtColor(imgs['edges'], cv2.COLOR_GRAY2RGB))
#axes[1].imshow(cv2.cvtColor(imgs['hough_lines'], cv2.COLOR_BGR2RGB))
#axes[2].imshow(cv2.cvtColor(imgs['corners'], cv2.COLOR_BGR2RGB))
#axes[3].imshow(cv2.cvtColor(imgs['warp'], cv2.COLOR_BGR2RGB))
#plt.plot()

# Find text
word = "and"
original, warped = doc.findWord(word)

cv2.imwrite("%s/original_%s" % (out_path, filename), original)
cv2.imwrite("%s/warped_%s" % (out_path, filename), warped)

Using TensorFlow backend.
/usr/src/app/koda/edge/network.py:66: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  self.model = Model(input = inputs, output = conv10)


[[ 982.46106 2689.2314 ]
 [ 985.122   2716.4407 ]
 [1053.5833  2709.8882 ]
 [1050.6748  2682.7288 ]]
[[[ 982.46106 2689.2314 ]
  [ 985.122   2716.4407 ]
  [1053.5833  2709.8882 ]
  [1050.6748  2682.7288 ]]]
[[1616.3174 2640.881 ]
 [1621.0837 2666.7207]
 [1680.6384 2661.0115]
 [1675.6681 2635.2134]]
[[[1616.3174 2640.881 ]
  [1621.0837 2666.7207]
  [1680.6384 2661.0115]
  [1675.6681 2635.2134]]]
[[1529.8751 2845.973 ]
 [1534.4636 2873.277 ]
 [1593.7544 2867.4285]
 [1588.9569 2840.168 ]]
[[[1529.8751 2845.973 ]
  [1534.4636 2873.277 ]
  [1593.7544 2867.4285]
  [1588.9569 2840.168 ]]]
[[ 758.8616  2951.953  ]
 [ 760.69727 2980.0503 ]
 [ 811.2189  2975.0452 ]
 [ 809.2004  2946.9856 ]]
[[[ 758.8616  2951.953  ]
  [ 760.69727 2980.0503 ]
  [ 811.2189  2975.0452 ]
  [ 809.2004  2946.9856 ]]]
[[1609.5209 2921.9397]
 [1614.4452 2949.7935]
 [1673.2483 2943.9233]
 [1668.1147 2916.1138]]
[[[1609.5209 2921.9397]
  [1614.4452 2949.7935]
  [1673.2483 2943.9233]
  [1668.1147 2916.1138]]]
[[ 714.16907 

True

<Figure size 1440x1440 with 0 Axes>